In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from TradingEnv import StockTradingEnv
from pybroker import YFinance
import pybroker
import pandas as pd
pybroker.enable_data_source_cache('yfinance')
from stable_baselines3.common.vec_env import DummyVecEnv


yfinance = YFinance()
df = yfinance.query(['AAPL'], start_date='3/1/2021', end_date='3/1/2022')
df['date'] = pd.to_datetime(df['date']).dt.date


# env_builder = lambda: gym.make('stocks-v0', df = df)
# env = DummyVecEnv([env_builder])
env = StockTradingEnv(df, initial_balance=100000, commission_fee=0.0001, slippage_cost=0.005)

# env = DummyVecEnv([env])

from stable_baselines3 import PPO

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=10_000)

model.save("ppo_aapl")

del model # remove to demonstrate saving and loading



In [5]:
model = PPO.load("ppo_aapl")

# vec_env = model.get_env()
obs = env.reset()
for i in range(len(df['adj_close'])):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = vec_env.step(action)


vec_env.render_all()

# observation = env.reset()
# for _ in range(1, len(df['adj_close'])):
#       # Display current state
#     action = env.action_space.sample()  # Random action for demonstration
#     observation, reward, done, truncated, info = env.step(action)
    
# env.render_all()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.